In [1]:
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust according to the output of your convolutional layers.
        self.fc2 = nn.Linear(128, 2)  # 2 output classes: occupied and vacant

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        # Max pool layer
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        # Max pool layer
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)

        # Fully connected layer 2
        out = self.fc2(out)
        return out


In [3]:
#Data prep
# Define a transform to resize and normalize the images, and then convert them to PyTorch tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert images to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values in the range [-1, 1]
])

# Define your parking spaces
parking_spaces = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']

# Path to your data
base_data_dir = '~/ParkEzAI/core_scripts/output/'

In [ ]:
# Prepare the model for each parking space
for parking_space in parking_spaces:
    data_dir = base_data_dir + parking_space

    # Create ImageFolder dataset
    dataset = ImageFolder(data_dir, transform=transform)

    # Train / test split and make data loaders
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Use the previously defined CNN class
    model = CNN()

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Number of epochs to train for
    num_epochs = 25

    # Move model to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # set the model to training mode
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            # Move data and labels to GPU if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Clear the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(train_loader.dataset)

        print('Parking Space: {}, Epoch {}, Loss: {:.4f}'.format(parking_space, epoch + 1, train_loss))

    print('Finished Training for parking space ', parking_space)

    # switch model to evaluation mode
    model.eval()

    correct = 0
    total = 0

    # no gradient computation during evaluation
    with torch.no_grad():
        for images, labels in test_loader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test accuracy for {parking_space}: {100 * correct / total}%')
    print(f'Correct / Total for {parking_space}: {correct} / {total}')

    # save the model after training
    torch.save(model.state_dict(), f"{parking_space}_model.pth")

Parking Space: A1, Epoch 1, Loss: 32.4578
Parking Space: A1, Epoch 2, Loss: 4.2102
Parking Space: A1, Epoch 3, Loss: 0.4862
Parking Space: A1, Epoch 4, Loss: 0.6069
Parking Space: A1, Epoch 5, Loss: 0.8347
Parking Space: A1, Epoch 6, Loss: 0.1953
Parking Space: A1, Epoch 7, Loss: 0.4293
Parking Space: A1, Epoch 8, Loss: 0.0278
Parking Space: A1, Epoch 9, Loss: 0.0698
Parking Space: A1, Epoch 10, Loss: 0.0040
Parking Space: A1, Epoch 11, Loss: 0.1482
Parking Space: A1, Epoch 12, Loss: 0.0388
Parking Space: A1, Epoch 13, Loss: 0.0441
Parking Space: A1, Epoch 14, Loss: 0.0050
Parking Space: A1, Epoch 15, Loss: 0.0000
Parking Space: A1, Epoch 16, Loss: 0.0001
Parking Space: A1, Epoch 17, Loss: 0.0003
Parking Space: A1, Epoch 18, Loss: 0.0197
Parking Space: A1, Epoch 19, Loss: 0.0000
Parking Space: A1, Epoch 20, Loss: 0.0000
Parking Space: A1, Epoch 21, Loss: 0.0573
Parking Space: A1, Epoch 22, Loss: 0.0000
Parking Space: A1, Epoch 23, Loss: 0.0006
Parking Space: A1, Epoch 24, Loss: 0.0213


Parking Space: B5, Epoch 6, Loss: 0.0000
Parking Space: B5, Epoch 7, Loss: 0.3481
Parking Space: B5, Epoch 8, Loss: 0.0000
Parking Space: B5, Epoch 9, Loss: 0.0000
Parking Space: B5, Epoch 10, Loss: 0.0000
Parking Space: B5, Epoch 11, Loss: 0.0000
Parking Space: B5, Epoch 12, Loss: 0.0000
Parking Space: B5, Epoch 13, Loss: 0.0000
Parking Space: B5, Epoch 14, Loss: 0.0000
Parking Space: B5, Epoch 15, Loss: 0.0000
Parking Space: B5, Epoch 16, Loss: 0.0000
Parking Space: B5, Epoch 17, Loss: 0.0000
Parking Space: B5, Epoch 18, Loss: 0.0000
Parking Space: B5, Epoch 19, Loss: 0.0000
Parking Space: B5, Epoch 20, Loss: 0.0000
Parking Space: B5, Epoch 21, Loss: 0.0000
Parking Space: B5, Epoch 22, Loss: 0.0000
Parking Space: B5, Epoch 23, Loss: 0.0000
Parking Space: B5, Epoch 24, Loss: 0.0000
Parking Space: B5, Epoch 25, Loss: 0.0000
Finished Training for parking space  B5
Test accuracy for B5: 100.0%
Correct / Total for B5: 39 / 39
Parking Space: B6, Epoch 1, Loss: 32.4792
Parking Space: B6, Ep

In [ ]:
# Understand what the dataset the looks like
# image, label = dataset[100]
# image = image.permute(1, 2, 0)  # Change from (C, H, W) to (H, W, C)

# plt.imshow(image)
# plt.show()
# print(label)